---
downloads:
  - file: output/fastqc/raw/SRR10971381_1_fastqc.html
    title: FASTQC report #1
    id: fastqc-report-1
  - file: output/fastqc/raw/SRR10971381_2_fastqc.html
    title: FASTQC report #2
    id: fastqc-report-2
---

# Wu _et al._ (2020)

:::{seealso} Overview
:class: dropdown
:name: Overview

**Title**: A new coronavirus associated with human respiratory disease in China

**Authors**: Fan Wu, Su Zhao, Bin Yu, Yan-Mei Chen, Wen Wang, Zhi-Gang Song, Yi Hu, Zhao-Wu Tao, Jun-Hua Tian, Yuan-Yuan Pei, Ming-Li Yuan, Yu-Ling Zhang, Fa-Hui Dai, Yi Liu, Qi-Min Wang, Jiao-Jiao Zheng, Lin Xu, Edward C Holmes, Yong-Zhen Zhang

**Publication Year**: 2020

**Journal**: Nature

**DOI**: https://doi.org/10.1038/s41586-020-2008-3
:::

## Abstract

Emerging infectious diseases, such as severe acute respiratory syndrome (SARS) and Zika virus disease, present a major threat to public health1–3. Despite intense research efforts, how, when and where new diseases appear are still a source of considerable uncertainty. A severe respiratory disease was recently reported in Wuhan, Hubei province, China. As of 25 January 2020, at least 1,975 cases had been reported since the first patient was hospitalized on 12 December 2019. Epidemiological investigations have suggested that the outbreak was associated with a seafood market in Wuhan. Here we study a single patient who was a worker at the market and who was admitted to the Central Hospital of Wuhan on 26 December 2019 while experiencing a severe respiratory syndrome that included fever, dizziness and a cough. Metagenomic RNA sequencing4 of a sample of bronchoalveolar lavage fluid from the patient identified a new RNA virus strain from the family Coronaviridae, which is designated here ‘WH-Human 1’ coronavirus (and has also been referred to as ‘2019-nCoV’). Phylogenetic analysis of the complete viral genome (29,903 nucleotides) revealed that the virus was most closely related (89.1% nucleotide similarity) to a group of SARS-like coronaviruses (genus Betacoronavirus, subgenus Sarbecovirus) that had previously been found in bats in China5. This outbreak highlights the ongoing ability of viral spill-over from animals to cause severe disease in humans.

## Environment Setup

Create a config file for the conda environment:
```{code} yaml
:filename: env.yml
name: wu2020
channels:
  - bioconda
  - conda-forge
dependencies:
  - fastqc
  - megahit
  - seqkit
  - sra-tools
  - trimmomatic
  - pip:
    - bio==1.7.1
```

Initialize the environment and install dependencies:
```{code-cell} bash
micromamba create -f env.yml
```

## Project Structure

In [2]:
%%bash

# For storing sequencing reads.
mkdir -p reads

# For storing genome records.
mkdir -p genomes

# For storing statistics in text format.
mkdir -p stats

# For building BLAST databases.
mkdir -p db

# For storing FastQC output.
mkdir -p output/fastqc

In [5]:
!lsd --tree --depth 2 -I ".*" -I "*.ipynb" -I "*.stats" -I "all_virus_genomes" --color never --icon never

.
├── db
│   ├── contigs
│   └── virus_nt_db
├── genomes
│   └── all_virus_genomes.fna
├── output
│   ├── blast.result
│   ├── blast.results
│   ├── draft_genome.fa
│   ├── fastqc
│   └── megahit
├── reads
│   ├── raw
│   └── trimmed
└── stats


## Workflow

### 1. Retrieve Project Metadata

In [6]:
%%bash

# Accession for the viral reads.
SRR=SRR10971381

# Fetch sample metadata.
bio search ${SRR}

[
    {
        "run_accession": "SRR10971381",
        "sample_accession": "SAMN13922059",
        "sample_alias": "Human-BALF",
        "sample_description": "Keywords: GSC:MIxS MIMS:5.0",
        "first_public": "2020-02-05",
        "country": "China: Wuhan",
        "scientific_name": "human lung metagenome",
        "fastq_bytes": "1160191847;1216048922",
        "base_count": "8031043214",
        "read_count": "28282964",
        "library_name": "1",
        "library_strategy": "RNA-Seq",
        "library_source": "METATRANSCRIPTOMIC",
        "library_layout": "PAIRED",
        "instrument_platform": "ILLUMINA",
        "instrument_model": "Illumina MiniSeq",
        "study_title": "Complete genome of a novel coronavirus associated with severe human respiratory disease in Wuhan, China",
        "fastq_url": [
            "https://ftp.sra.ebi.ac.uk/vol1/fastq/SRR109/081/SRR10971381/SRR10971381_1.fastq.gz",
            "https://ftp.sra.ebi.ac.uk/vol1/fastq/SRR109/081/SRR10971381

### 2. Download FASTQ Files

In [47]:
%%bash

# Create a subdirectory for raw reads.
mkdir -p reads/raw

# Retrieve read data from SRA.
fastq-dump --split-files --origfmt --outdir reads/raw SRR10971381 

Read 28282964 spots for SRR10971381
Written 28282964 spots for SRR10971381


Get an overview of the FASTQ files:

In [48]:
!seqkit stats --quiet reads/raw/*

file                           format  type    num_seqs        sum_len  min_len  avg_len  max_len
reads/raw/SRR10971381_1.fastq  FASTQ   DNA   28,282,964  4,017,125,680       35      142      151
reads/raw/SRR10971381_2.fastq  FASTQ   DNA   28,282,964  4,013,917,534       35    141.9      151


Inspect the first two forward reads:

In [49]:
!head -8 reads/raw/SRR10971381_1.fastq

@1
CAAAGTCAAGCTCCCTTCTGCCTTTACACTCTTCGAGCGATTTCCGTCCGCTCTGAGGGAACCTTTGGGCGCCTCCGTTACTCTTTTGGAGGCGACCGCCCCAGTCAAACTGCCCGCCTAAGACTGTCCGGCCGGTCNTTACTCGGCNCGT
+1
AAFFAF/FFFFFFFFAFFFFFFFFFFFFFFF/FFFFFFF/AFFFFFFFFFFFFAF/FFF//FF=FA/FFAFFFFFF/FF/FAFFF/AFFFAF6FFF//FFAFFAFFFFFFFFFFFFFFFFAFFFAA=A/FFFAFFFF#6AFF6F=FF#=FF
@2
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
+2
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


### 3. Quality Control

Generate a quality report for the raw reads using `fastqc`:

:::{aside} FASTQC Reports

- [Report #1](output/fastqc/raw/SRR10971381_1_fastqc.html)
- [Report #2](output/fastqc/raw/SRR10971381_2_fastqc.html)

:::

In [50]:
%%bash

# Store report for raw reads here.
mkdir -p output/fastqc/raw

# Generate report.
fastqc -o output/fastqc/raw --svg reads/raw/*.fastq 

null
null


Started analysis of SRR10971381_1.fastq
Approx 5% complete for SRR10971381_1.fastq
Approx 10% complete for SRR10971381_1.fastq
Approx 15% complete for SRR10971381_1.fastq
Approx 20% complete for SRR10971381_1.fastq
Approx 25% complete for SRR10971381_1.fastq
Approx 30% complete for SRR10971381_1.fastq
Approx 35% complete for SRR10971381_1.fastq
Approx 40% complete for SRR10971381_1.fastq
Approx 45% complete for SRR10971381_1.fastq
Approx 50% complete for SRR10971381_1.fastq
Approx 55% complete for SRR10971381_1.fastq
Approx 60% complete for SRR10971381_1.fastq
Approx 65% complete for SRR10971381_1.fastq
Approx 70% complete for SRR10971381_1.fastq
Approx 75% complete for SRR10971381_1.fastq
Approx 80% complete for SRR10971381_1.fastq
Approx 85% complete for SRR10971381_1.fastq
Approx 90% complete for SRR10971381_1.fastq
Approx 95% complete for SRR10971381_1.fastq


Analysis complete for SRR10971381_1.fastq


Started analysis of SRR10971381_2.fastq
Approx 5% complete for SRR10971381_2.fastq
Approx 10% complete for SRR10971381_2.fastq
Approx 15% complete for SRR10971381_2.fastq
Approx 20% complete for SRR10971381_2.fastq
Approx 25% complete for SRR10971381_2.fastq
Approx 30% complete for SRR10971381_2.fastq
Approx 35% complete for SRR10971381_2.fastq
Approx 40% complete for SRR10971381_2.fastq
Approx 45% complete for SRR10971381_2.fastq
Approx 50% complete for SRR10971381_2.fastq
Approx 55% complete for SRR10971381_2.fastq
Approx 60% complete for SRR10971381_2.fastq
Approx 65% complete for SRR10971381_2.fastq
Approx 70% complete for SRR10971381_2.fastq
Approx 75% complete for SRR10971381_2.fastq
Approx 80% complete for SRR10971381_2.fastq
Approx 85% complete for SRR10971381_2.fastq
Approx 90% complete for SRR10971381_2.fastq
Approx 95% complete for SRR10971381_2.fastq


Analysis complete for SRR10971381_2.fastq


Trim adapters and filiter out low quality reads using `trimmomatic`:

In [51]:
%%bash

# SRA accession.
SRR=SRR10971381

# Path to pair-end reads.
R1=reads/raw/${SRR}_1.fastq
R2=reads/raw/${SRR}_2.fastq

# Directory for storing trimmed reads.
mkdir -p reads/trimmed

# Perform trimmimng and filtering.
trimmomatic PE ${R1} ${R2} \
  reads/trimmed/${SRR}_1{,un}.trimmed.fastq reads/trimmed/${SRR}_2{,un}.trimmed.fastq \
  -summary stats/${SRR}.trimmomatic.stats \
  SLIDINGWINDOW:4:30

TrimmomaticPE: Started with arguments:
 reads/raw/SRR10971381_1.fastq reads/raw/SRR10971381_2.fastq reads/trimmed/SRR10971381_1.trimmed.fastq reads/trimmed/SRR10971381_1un.trimmed.fastq reads/trimmed/SRR10971381_2.trimmed.fastq reads/trimmed/SRR10971381_2un.trimmed.fastq -summary stats/SRR10971381.trimmomatic.stats SLIDINGWINDOW:4:30
Quality encoding detected as phred33
Input Read Pairs: 28282964 Both Surviving: 5304391 (18.75%) Forward Only Surviving: 8220297 (29.06%) Reverse Only Surviving: 226735 (0.80%) Dropped: 14531541 (51.38%)
TrimmomaticPE: Completed successfully


Check how many reads were trimmed:

In [52]:
!cat stats/SRR10971381.trimmomatic.stats

Input Read Pairs: 28282964
Both Surviving Reads: 5304391
Both Surviving Read Percent: 18.75
Forward Only Surviving Reads: 8220297
Forward Only Surviving Read Percent: 29.06
Reverse Only Surviving Reads: 226735
Reverse Only Surviving Read Percent: 0.80
Dropped Reads: 14531541
Dropped Read Percent: 51.38


Generate FASTQC report for trimmed reads:

:::{aside} FASTQC Reports

- [Report #1](output/fastqc/trimmed/SRR10971381_1.trimmed_fastqc.html)
- [Report #2](output/fastqc/trimmed/SRR10971381_2.trimmmed_fastqc.html)

:::

In [53]:
%%bash

# Store report for trimmed reads here.
mkdir -p output/fastqc/trimmed

# Generate report.
fastqc -o output/fastqc/trimmed --svg reads/trimmed/*.fastq 

null
null


Started analysis of SRR10971381_1.trimmed.fastq


null
null


Approx 5% complete for SRR10971381_1.trimmed.fastq
Approx 10% complete for SRR10971381_1.trimmed.fastq
Approx 15% complete for SRR10971381_1.trimmed.fastq
Approx 20% complete for SRR10971381_1.trimmed.fastq
Approx 25% complete for SRR10971381_1.trimmed.fastq
Approx 30% complete for SRR10971381_1.trimmed.fastq
Approx 35% complete for SRR10971381_1.trimmed.fastq
Approx 40% complete for SRR10971381_1.trimmed.fastq
Approx 45% complete for SRR10971381_1.trimmed.fastq
Approx 50% complete for SRR10971381_1.trimmed.fastq
Approx 55% complete for SRR10971381_1.trimmed.fastq
Approx 60% complete for SRR10971381_1.trimmed.fastq
Approx 65% complete for SRR10971381_1.trimmed.fastq
Approx 70% complete for SRR10971381_1.trimmed.fastq
Approx 75% complete for SRR10971381_1.trimmed.fastq
Approx 80% complete for SRR10971381_1.trimmed.fastq
Approx 85% complete for SRR10971381_1.trimmed.fastq
Approx 90% complete for SRR10971381_1.trimmed.fastq
Approx 95% complete for SRR10971381_1.trimmed.fastq


Analysis complete for SRR10971381_1.trimmed.fastq


...

Analysis complete for SRR10971381_1un.trimmed.fastq


Started analysis of SRR10971381_2.trimmed.fastq
Approx 5% complete for SRR10971381_2.trimmed.fastq
Approx 10% complete for SRR10971381_2.trimmed.fastq
Approx 15% complete for SRR10971381_2.trimmed.fastq
Approx 20% complete for SRR10971381_2.trimmed.fastq
Approx 25% complete for SRR10971381_2.trimmed.fastq
Approx 30% complete for SRR10971381_2.trimmed.fastq
Approx 35% complete for SRR10971381_2.trimmed.fastq
Approx 40% complete for SRR10971381_2.trimmed.fastq
Approx 45% complete for SRR10971381_2.trimmed.fastq
Approx 50% complete for SRR10971381_2.trimmed.fastq
Approx 55% complete for SRR10971381_2.trimmed.fastq
Approx 60% complete for SRR10971381_2.trimmed.fastq
Approx 65% complete for SRR10971381_2.trimmed.fastq
Approx 70% complete for SRR10971381_2.trimmed.fastq
Approx 75% complete for SRR10971381_2.trimmed.fastq
Approx 80% complete for SRR10971381_2.trimmed.fastq
Approx 85% complete for SRR10971381_2.trimmed.fastq
Approx 90% complete for SRR10971381_2.trimmed.fastq
Approx 95% comple

Analysis complete for SRR10971381_2.trimmed.fastq


Started analysis of SRR10971381_2un.trimmed.fastq
Approx 5% complete for SRR10971381_2un.trimmed.fastq
Approx 10% complete for SRR10971381_2un.trimmed.fastq
Approx 15% complete for SRR10971381_2un.trimmed.fastq
Approx 20% complete for SRR10971381_2un.trimmed.fastq
Approx 25% complete for SRR10971381_2un.trimmed.fastq
Approx 30% complete for SRR10971381_2un.trimmed.fastq
Approx 35% complete for SRR10971381_2un.trimmed.fastq
Approx 40% complete for SRR10971381_2un.trimmed.fastq
Approx 45% complete for SRR10971381_2un.trimmed.fastq
Approx 50% complete for SRR10971381_2un.trimmed.fastq
Approx 55% complete for SRR10971381_2un.trimmed.fastq
Approx 60% complete for SRR10971381_2un.trimmed.fastq
Approx 65% complete for SRR10971381_2un.trimmed.fastq
Approx 70% complete for SRR10971381_2un.trimmed.fastq
Approx 75% complete for SRR10971381_2un.trimmed.fastq
Approx 80% complete for SRR10971381_2un.trimmed.fastq
Approx 85% complete for SRR10971381_2un.trimmed.fastq
Approx 90% complete for SRR109713

Analysis complete for SRR10971381_2un.trimmed.fastq


### 4. Genome Assembly

Generate a draft assembly using `megahit`:

In [55]:
%%bash

# SRA accession.
SRR=SRR10971381

# Path to trimmed reads.
R1=reads/trimmed/${SRR}_1.trimmed.fastq
R2=reads/trimmed/${SRR}_2.trimmed.fastq

# Perform assembly.
megahit -1 ${R1} -2 ${R2} -o output/megahit

2025-02-07 22:53:07 - MEGAHIT v1.2.9
2025-02-07 22:53:07 - Using megahit_core with POPCNT and BMI2 support
2025-02-07 22:53:07 - Convert reads to binary library
2025-02-07 22:53:11 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/dagsdags/gh-repos/redo/wu2020/reads/trimmed/SRR10971381_1.trimmed.fastq,/home/dagsdags/gh-repos/redo/wu2020/reads/trimmed/SRR10971381_2.trimmed.fastq): pe, 10608782 reads, 151 max length'
2025-02-07 22:53:11 - b'INFO  utils/utils.h                 :  152 - Real: 3.3677\tuser: 3.1343\tsys: 0.8476\tmaxrss: 281940'
2025-02-07 22:53:11 - k-max reset to: 141 
2025-02-07 22:53:11 - Start assembly. Number of CPU threads 16 
2025-02-07 22:53:11 - k list: 21,29,39,59,79,99,119,141 
2025-02-07 22:53:11 - Memory used: 26183522304
2025-02-07 22:53:11 - Extract solid (k+1)-mers for k = 21 
2025-02-07 22:53:33 - Build graph for k = 21 
2025-02-07 22:53:39 - Assemble contigs from SdBG for k = 21
2025-02-07 22:54:05 - Local assembly for k = 21
2025-02-07 22:54:09

Get an overview of the generated contigs:

In [56]:
!seqkit stats output/megahit/final.contigs.fa

file                             format  type  num_seqs    sum_len  min_len  avg_len  max_len
output/megahit/final.contigs.fa  FASTA   DNA      7,599  3,777,357      200    497.1   29,874


### 5. Virus Identification

Retrieve all viral nucleotide records from RefSeq and build a BLAST database:

In [ ]:
%%bash

# Download all virus nt records.
genome_updater.sh -d "refseq" -g "viral" -c "na" \
  -f "genomic.fna.gz" -o "all_virus_genomes" -t 8

# Concatenate all FASTA files into a single file.
zcat all_virus_genomes/2025-02-08_00-06-28/files/*.fna.gz > all_virus_genomes.fna

# Create BLAST db for viruses.
mkdir -p virus_nt_db
makeblastdb -in all_virus_genomes.fna -out virus_nt_db/virus -dbtype nucl -parse_seqids

Generate a BLAST database from the assembled contigs:

In [57]:
%%bash

# A directory for storing BLAST databases.
mkdir -p db

makeblastdb -in output/megahit/final.contigs.fa -out db/contigs -dbtype nucl -parse_seqids



Building a new DB, current time: 02/07/2025 23:02:25
New DB name:   /home/dagsdags/gh-repos/redo/wu2020/db/contigs
New DB title:  output/megahit/final.contigs.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 7599 sequences in 0.10007 seconds.




Sort the contigs by length and extract the ID of the longest one:

In [74]:
!blastdbcmd -db db/contigs -entry all -outfmt "%l %a" | sort -k1 -r -n | head -1 | cut -d " " -f2

k141_1553
sort: write failed: 'standard output': Broken pipe
sort: write error


Extract the sequence of the largest contig and save it as a separate FASTA file:

In [78]:
!blastdbcmd -db db/contigs -entry K141_1553 > output/draft_genome.fa

In [79]:
!cat output/draft_genome.fa | head

>k141_1553 flag=1 multi=109.0000 len=29874
TGATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT
AAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACA
GGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGT
TTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTT
GCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAAC
ATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATC
AAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGG
TCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTA
AGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACT


Query contig against the locally built viral database:

In [84]:
!blastn -db virus_nt_db/virus -query output/draft_genome.fa -outfmt 6 > output/blast.results

In [88]:
!cat output/blast.result | head -30 | tail -13

                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isola...  55160   0.0   
NC_004718.3 SARS coronavirus Tor2, complete genome                    15175   0.0   
NC_014470.1 Bat coronavirus BM48-31/BGR/2008, complete genome         13461   0.0   
NC_025217.1 Bat Hp-betacoronavirus/Zhejiang2013, complete genome      1629    0.0   
NC_034440.1 Bat coronavirus isolate PREDICT/PDF-2180, complete ge...  758     0.0   
NC_006577.2 Human coronavirus HKU1, complete genome                   496     8e-137
NC_048214.1 Duck coronavirus isolate DK/GD/27/2014, complete genome   272     1e-69 
NC_048211.1 Wencheng Sm shrew coronavirus isolate Xingguo-74 ORF1...  167     7e-38 
NC_035191.1 Wencheng Sm shrew coronavirus isolate Xingguo-101 ORF...  161     3e-36 
NC_016991.1 White-eye coronavirus HKU16, complete genome             